In [1]:
import main
main.create_csv_from_las()

Convert224ST4.LAS
Convert238ST4.LAS


Convert274ST4.LAS
Convert300.LAS


Convert301.LAS
Convert302.LAS


Convert304.LAS


Convert305.LAS
Convert306.LAS


...

Convert841.LAS
Convert850.LAS


Convert852.LAS
Convert853.LAS


Convert855.LAS
Convert856.LAS


Convert857.LAS
Convert859.LAS


end save csv


In [1]:
import numpy as np
import pandas as pd
import main

las_data = pd.read_csv(main.csv_out_file, delimiter=';')
water_content_data = pd.read_csv(main.data_dir + "\\res.csv", delimiter=';')

water_content_data = water_content_data.rename(columns={'Скважина': main.keys_dict[main.kid_well]})
water_content_data = water_content_data.rename(columns={' Начальная обводненность ': 'wc'})
del water_content_data['Дата запуска']

result = pd.merge(las_data, water_content_data, on=main.keys_dict[main.kid_well])
print(len(result['well_name'].value_counts()))

173


In [70]:
result.head()
result['lit'].describe()

count    390001.000000
mean          0.178638
std           0.383049
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: lit, dtype: float64

In [295]:
step_h_count = 5
data_well_list = []
# Выделяем даные для одной скважины
data_well = result[result['well_name'] == '300']

# высчитываем для нее характерисктики шага по глубине
start_h = data_well[:1]["STRT"]
end_h = data_well[:1]["STOP"]
start_h = float(start_h)
end_h = float(end_h)
h_step = (float(end_h) - float(start_h))/step_h_count
print(float(start_h), float(end_h), h_step)
temp_start_h = start_h
index = 1
data_for_analize = []
while temp_start_h < end_h:
    str_index = str(index)
    temp_end_h = temp_start_h + h_step
    # выделяем для конкретного участка по глубине
    data_well_by_h = data_well[(data_well['DEPT'] >= temp_start_h) & (data_well['DEPT'] < temp_end_h)]
    columns_to_show=['aps', 'rp', 'kp', 'kgl', 'kpr', 'kvo', 'kng', 'lit', 'satur', 'wc']
    # расчитываем среднее
    data_well_by_h = data_well_by_h.pivot_table(['aps', 'rp', 'kp', 'kgl', 'kpr', 'kvo', 'kng', 'lit', 'satur'], ['well_name'] ,aggfunc='mean')
    data_well_by_h = data_well_by_h.rename(index=str, columns={"aps": 'aps' + str_index, 'rp': 'rp' + str_index, 'kp': 'kp' + str_index, 'kgl': 'kgl' + str_index, 
                                                                'kpr': 'kpr' + str_index, 'kvo': 'kvo' + str_index, 'kng': 'kng' + str_index, 'lit': 'lit'  + str_index,
                                                                'satur': 'satur' + str_index})
    data_well_list.append(data_well_by_h)
    temp_start_h = temp_end_h
    index = index + 1
    # print(temp_start_h, temp_end_h, end_h)

3114.0 3363.6 49.91999999999998


In [296]:
# делаем ключи и праметры плоскими (размерности 1)
key_list = list([])
for elem in data_well_list:
    key_list.extend(list(elem))
values_list = np.vstack(data_well_list)
values_list = values_list.ravel()
values_list = values_list.tolist()
values_data = [values_list]
res_frame = pd.DataFrame(data = values_data,
                 columns=key_list)
res_frame['well_name'] = '300'
res_frame

,aps1,kgl1,kng1,kp1,kpr1,kvo1,lit1,rp1,satur1,aps2,...,aps5,kgl5,kng5,kp5,kpr5,kvo5,lit5,rp5,satur5,well_name
0,0.089075,0.209994,0.0,0.108358,4.087861,0.960497,0.052,11.390173,0.104,0.018302,...,0.406791,0.23854,0.0,0.095726,2.349751,1.0,0.24,7.480597,0.48,300
